<a href="https://colab.research.google.com/github/timoNieke/book-recommender-KNN/blob/main/fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [80]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [140]:

# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2023-03-13 21:17:03--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.6’

book-crossings.zip. 100%[===================>]  24.88M  --.-KB/s    in 0.1s    

2023-03-13 21:17:03 (237 MB/s) - ‘book-crossings.zip.6’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [176]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [177]:
book_counts = df_ratings['isbn'].value_counts()
popular_books = book_counts[book_counts >= 100].index.tolist()
df_popular_books = df_books[df_books['isbn'].isin(popular_books)]

In [178]:
user_counts = df_ratings['user'].value_counts()
users_with_enough_ratings = user_counts[user_counts >= 200].index
users_with_enough_ratings

Int64Index([ 11676, 198711, 153662,  98391,  35859, 212898, 278418,  76352,
            110973, 235105,
            ...
             28634,  59727, 268622, 188951, 225595,  83671, 252827,  99955,
             36554,  26883],
           dtype='int64', length=905)

In [179]:
merged_df = pd.merge(df_ratings.loc[df_ratings["user"].isin(users_with_enough_ratings)],
              df_popular_books[["isbn"]], 
              on="isbn",
              how="inner")
merged_df

,user,isbn,rating
0,277427,002542730X,10.0
1,3363,002542730X,0.0
2,11676,002542730X,6.0
3,12538,002542730X,10.0
4,13552,002542730X,0.0
...,...,...,...
49512,238864,0399149325,0.0
49513,251843,0399149325,1.0
49514,253821,0399149325,0.0
49515,265115,0399149325,0.0


In [180]:
# create pivot table
ratings_pivot = merged_df.pivot_table(index='isbn', columns='user', values='rating').fillna(0)
ratings_pivot

user,254,2276,2766,2977,3363,4017,4385,6242,6251,6323,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
isbn,,,,,,,,,,,,,,,,,,,,,
002542730X,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0
0060008032,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0060096195,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
006016848X,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0060173289,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1573227331,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1573229326,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1573229571,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [181]:
# create sparse matrix
X = csr_matrix(ratings_pivot.values)
X

<727x888 sparse matrix of type '<class 'numpy.float32'>'
	with 12548 stored elements in Compressed Sparse Row format>

In [182]:
# build model
model = NearestNeighbors(metric='cosine', algorithm='brute')
model.fit(X)

NearestNeighbors(algorithm='brute', metric='cosine')

In [206]:
def get_recommends(title):

  # get book features
  book = df_popular_books.loc[df_popular_books['title'] == title]
  book_isbn = book['isbn'].iloc[0]
  book_features = ratings_pivot.loc[book_isbn]

  # find k-nearest neighbors
  model = NearestNeighbors(n_neighbors=6, algorithm='brute', metric='cosine')
  model.fit(ratings_pivot)
  distances, indices = model.kneighbors([book_features])

  # get recommended books
  recommended_books = []
  for i in range(len(indices[0])-1, 0, -1):
      recommended_isbn = ratings_pivot.iloc[indices[0][i]].name
      recommended_title = df_popular_books.loc[df_popular_books['isbn'] == recommended_isbn]['title'].iloc[0]
      recommended_books.append((recommended_title, distances[0][i]))

  return (book['title'].iloc[0], recommended_books)

In [205]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

("Where the Heart Is (Oprah's Book Club (Paperback))", [("I'll Be Seeing You", 0.8016211), ('The Weight of Water', 0.77085835), ('The Surgeon', 0.7699411), ('I Know This Much Is True', 0.7677075), ('The Lovely Bones: A Novel', 0.7234864)])
You passed the challenge! 🎉🎉🎉🎉🎉
